In [ ]:
import time
import pandas as pd
from pybit.unified_trading import HTTP
from datetime import datetime

# === CONFIGURATION ===
# TO-DO: put these in .env
API_KEY = "YOUR_TESTNET_API_KEY"
API_SECRET = "YOUR_TESTNET_API_SECRET"
SYMBOL = "BTCUSDT"
INITIAL_TRADE_QTY = 0.001
MAX_TRADES = 5
MULTIPLIER = 2
TP_PCT = 0.005
SL_PCT = 0.005
TRADE_INTERVAL = 10  # seconds

# === INITIALIZE CLIENT ===
session = HTTP(
    testnet=True,
    api_key=API_KEY,
    api_secret=API_SECRET
)

# === STATE ===
position_qty = 0
entry_price = 0
trade_count = 0
trade_log = []

def get_price():
    ticker = session.get_tickers(category="linear", symbol=SYMBOL)
    return float(ticker['result']['list'][0]['lastPrice'])

def place_order(side, qty):
    response = session.place_order(
        category="linear",
        symbol=SYMBOL,
        side=side,
        order_type="Market",
        qty=qty,
        time_in_force="GoodTillCancel"
    )
    print(f"[ORDER] {side} {qty} {SYMBOL}")
    return response

def log_trade(price, side, qty):
    trade_log.append({
        "timestamp": datetime.utcnow().isoformat(),
        "side": side,
        "qty": qty,
        "price": price
    })

def print_stats():
    if not trade_log:
        print("No trades yet.")
        return

    df = pd.DataFrame(trade_log)
    print("\n=== Trade Summary ===")
    print(df.tail(5))
    print(f"Total Trades: {len(df)}")

def martingale_trade():
    global position_qty, entry_price, trade_count

    price = get_price()

    if trade_count == 0:
        qty = INITIAL_TRADE_QTY
        side = "Buy"
        entry_price = price
        position_qty += qty
    else:
        # Assume price went against us: Buy more at worse price
        qty = INITIAL_TRADE_QTY * (MULTIPLIER ** trade_count)
        side = "Buy"
        position_qty += qty
        entry_price = (entry_price * (position_qty - qty) + price * qty) / position_qty

    place_order(side, qty)
    log_trade(price, side, qty)
    trade_count += 1

def should_exit(price):
    if position_qty == 0:
        return False

    gain = (price - entry_price) / entry_price
    return gain >= TP_PCT or gain <= -SL_PCT

def reset_position():
    global position_qty, entry_price, trade_count
    side = "Sell"
    place_order(side, position_qty)
    log_trade(get_price(), side, position_qty)
    position_qty = 0
    entry_price = 0
    trade_count = 0
    print("Position closed due to TP/SL")

# === MAIN LOOP ===
try:
    while True:
        current_price = get_price()

        if position_qty == 0:
            martingale_trade()
        elif should_exit(current_price):
            reset_position()
        elif trade_count < MAX_TRADES:
            martingale_trade()

        print_stats()
        time.sleep(TRADE_INTERVAL)

except KeyboardInterrupt:
    print("Exiting... Saving log.")
    df = pd.DataFrame(trade_log)
    df.to_csv("trade_log.csv", index=False)
    print("Trade history saved to trade_log.csv.")


ModuleNotFoundError: No module named 'pybit'